In [ ]:
### Detect the faces in the video and classify whether it is a crying baby
import cv2
import numpy as np
import tensorflow as tf
import detect_imgs_CryingBaby_RL as detface # import face_detector

In [2]:
# Load the best crying baby classifation model
best_model_path = 'Model.epoch_119-val_acc_0.8795.h5'
model = tf.keras.models.load_model(best_model_path)

# Load the video
file_path = "baby1_640*480"
video_path = "videos/" + file_path + '.mp4'
cap = cv2.VideoCapture(video_path)  # capture from video
# cap = cv2.VideoCapture(0)  # capture from camera

print((int(cap.get(3)), int(cap.get(4))))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter("videos/Result/Result_"+ file_path + '.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Face detection and classification
while True:
    _, orig_image = cap.read()
    if orig_image is None:
        print("No Video.")
        break
        
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
   
    # Detect faces in video image, crop, and classify
    
    # Use CV2 Cascade to detect faces
    #face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    #faces = face_cascade.detectMultiScale(image, scaleFactor=1.08, minNeighbors=4, minSize=(30, 30))    
    #if len(faces) > 0:
    #  faces = (faces[:,0],faces[:,1],faces[:,0]+faces[:,2],faces[:,1]+faces[:,3])#Change from [x,y,w,h] to [xlt, ylt, xrb, yrb]
    #   faces = np.transpose(np.hstack([faces]))
 
    # Use Ultra Light Fast Generic Face Detector 1MB to detect faces
    # https://github.com/Linzaer/Ultra-Light-Fast-Generic-Face-Detector-1MB
    faces = detface.face_detector(image)
    
    for xlt,ylt,xrb,yrb in faces:
        box = [xlt, ylt, xrb, yrb]
        corners = np.rint(box).astype(int)
        corners = np.maximum(corners, [0,0,0,0])
        image_cut = image[corners[1]: corners[3], corners[0]: corners[2],:].copy() # Cut image to get faces
        image_cut = cv2.resize(image_cut, (150, 150)) # Resize the image to (150,150)
        image_cut = image_cut/255.    # Nomalize the image to (0,1)   
        image_cut = np.expand_dims(image_cut, axis=0)
        classes = model.predict(image_cut, batch_size=1) # Classify the image
        
        #Display the deteced boxes
        cv2.rectangle(orig_image, (xlt, ylt), (xrb, yrb), (0, 255, 0), 4)
        
        #Display the classification results
        if classes[0]<0.5:
            cv2.putText(orig_image, "Crying",
                    (xlt, ylt - 10),
                     cv2.FONT_HERSHEY_SIMPLEX,
                     fontScale = 1,  
                     color = (0, 0, 255),
                     thickness = 2)  
        else:
            cv2.putText(orig_image, "Not Crying",
                    (xlt, ylt - 10),
                     cv2.FONT_HERSHEY_SIMPLEX,
                     fontScale = 1,  
                     color = (255, 0, 0),
                     thickness = 2)     
    
    # Save the processed video
    out.write(orig_image) # Must have the same image size as in out definition
    
    # Show the processed video
    orig_image = cv2.resize(orig_image, None, None, fx=0.8, fy=0.8)
    cv2.imshow('annotated', orig_image)
    if cv2.waitKey(1) & 0xFF == ord('q'): # Quit the show
       break

cap.release()
out.release()

# Close the video window
cv2.startWindowThread()
cv2.destroyAllWindows()
cv2.waitKey(1)

W1201 12:49:00.861091 140735969207168 deprecation.py:323] From /Users/ll/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1423: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1201 12:49:00.988350 140735969207168 deprecation.py:323] From /Users/ll/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


(640, 480)
No Video.


-1